In [60]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Beispielhafte Daten
# X = np.random.rand(100, 13)
# y = np.random.rand(100, 5)

# Deine echten Daten hier laden
X = ... # Dein Input-Datensatz (100, 13)
y = ... # Dein Output-Datensatz (100, 5)

In [61]:
import csv
import re

def read_csv_file(file_path):
    # Initialisiere eine leere Liste, um die bereinigten Daten zu speichern
    cleaned_data = []

    # Lese die CSV-Datei
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=';')  # Annahme: Semikolon als Trennzeichen
        for row in reader:
            # Verbinde die Zeilenelemente mit einem Komma, um das Trennzeichen zu vereinheitlichen
            unified_row = ','.join(row)
            # Ersetze mehrere aufeinander folgende Kommas durch ein einzelnes Komma
            unified_row = re.sub(r',+', ',', unified_row)
            # Teile die vereinheitlichte Zeile nach dem Komma auf
            split_row = unified_row.split(',')
            # Entferne die ersten zwei Parameter
            cleaned_row = split_row[1:]
            # Füge die bereinigte Zeile der Liste hinzu
            cleaned_data.append(cleaned_row)

    return cleaned_data

# Beispiel: Daten aus "input.csv" einlesen

In [62]:
data = pd.read_csv("initial_data.csv")
cleaned_data = read_csv_file("querys_ForcePush.csv")
df_queried = pd.DataFrame(cleaned_data[1:], columns=cleaned_data[0])
def add_data(data, queried_data):
    # add queried data (without cost) to initial data
    data = pd.concat([data, queried_data.iloc[:, :13]], axis=0)

    # data = data.append(queried_data, ignore_index=True)
    return data

data = add_data(data, df_queried)
data = data.reset_index().astype('Float32')

print(data)

#print(data)
#print(data.keys())

x = data[['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']]
y = data[['NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder']]

x_max = x.max()
x_min = x.min()
y_max = y.max()
y_min = y.min()

#print(str(y_min) + "\n:\n" + str(y_max))

y_range = y_max - y_min

print(y_range)

     index  Engine speed  Engine load  Railpressure  Air supply  Crank angle  \
0      0.0         700.0         7.33    500.899994   78.375298         1.91   
1      1.0         700.0        25.85    577.599976  118.067299         0.94   
2      2.0         700.0    46.669998    615.799988  144.355698         2.86   
3      3.0         700.0    69.550003    778.299988  174.177902         2.22   
4      4.0         700.0    94.330002    900.099976  213.314804        -1.72   
..     ...           ...          ...           ...         ...          ...   
108    8.0   2183.257812    47.490673   1444.360107  650.760376     4.574534   
109    9.0   1896.591187    97.303398   1351.508545  827.040039    -3.222039   
110   10.0   1199.474487   101.517929   1868.645874  393.788116    -0.035194   
111   11.0     903.48175    10.912282    960.143188  602.400879     4.090658   
112   12.0    989.338928    81.168571    798.997742  610.851624     4.238798   

     Intake pressure  Back pressure  In

In [63]:
# Datenaufteilung in Training und Test
from numpy import double


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)


# Datenstandardisierung
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# X_train = scaler_X.fit_transform(X_train)
# X_test = scaler_X.transform(X_test)

# y_train = scaler_y.fit_transform(y_train)
# y_test = scaler_y.transform(y_test)

x_train = np.array(X_train.values, dtype=float)
x_test = np.array(X_train.values, dtype=float)
y_test = np.array(y_test.values, dtype=float)
y_train = np.array(y_train.values, dtype=float)

# Umwandlung in PyTorch Tensors
X_train = torch.from_numpy(x_train)
X_test = torch.from_numpy(x_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)


In [64]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(8, 16)
        self.layer2 = nn.Linear(16, 12)
        self.layer3 = nn.Linear(12, 10)
        self.output = nn.Linear(10, 5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        print(x.type)
        mat = torch.diag(torch.tensor([1, 1, 1, 1, 1, 1, 1, 1])).to(torch.float64)
        print(mat.type)
        x = torch.matmul(x, mat)
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.output(x)
        return x

model = NeuralNet()

In [65]:
# Verlustfunktion und Optimierer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trainingsparameter
num_epochs = 300
batch_size = 3

# Daten in Batches aufteilen
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [66]:
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        # Forward-Pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward-Pass und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Modell speichern (optional)
torch.save(model.state_dict(), 'model.pth')

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

In [ ]:
model.eval()  # Setzt das Modell in den Evaluationsmodus
with torch.no_grad():  # Keine Gradientenberechnung
    y_pred = model(X_test)
    #test_loss = criterion(y_pred, y_test)
    #print(y_test)
    
    #print(f'Test Loss: {test_loss.item():.4f}')
    # y_pred = scaler_y.inverse_transform(y_pred)
    # y_test = scaler_y.inverse_transform(y_test)
    for pred, test in zip(y_pred, y_test):
        print(str(pred) + ":" + str(test))

[213.38683499   2.92700271  99.85273479   2.51839192  95.81970808]:[165.12383952   3.63098577 104.66582448   7.39466181  99.42012039]
[373.58386933   0.73865842  41.63340887 150.46435799  93.4080011 ]:[315.43017611   0.77328927  39.26986336   4.29842533  93.84214037]
[147.01234983   2.58613577  63.63875948   4.35376852  79.56792392]:[134.17139995   3.24507671  65.66739663  12.91363239  74.54426619]
[ 46.74416278   0.79590238  16.18592546 -23.72326107  54.3879884 ]:[50.60140561  0.53715259 16.30251046  2.32967088 53.86597464]
[11.80065251  0.2218374   6.53372202  2.2914528  49.58671435]:[22.09589589  0.19848541  3.68189797  1.95892306 45.77107311]
[92.12566554  1.8562298  35.33763786 36.85857762 61.55148618]:[94.23298579  2.83292102 38.50904867 10.53630152 59.95670698]
[ 83.04124059   2.65315541  34.65893193 -42.98971699  61.57223513]:[66.86942159  4.09588523 36.29146669 16.28325391 60.4979292 ]
[213.60988223   2.54992465  92.62367252 -11.96991153  96.15501912]:[187.19869988   2.8929777

In [ ]:
# build matrix of elementwise mean squared error

from sklearn.metrics import mean_squared_error


matrix = []
for pred, test in zip(y_pred, y_test):
    data_set_mse = []
    for p, t in zip(pred, test):
        #print(str(pred) + ":" + str(test))
        mse = mean_squared_error([p], [t])
        data_set_mse.append(mse)
    #print(data_set_mse)

    percentage_to_range = data_set_mse / y_max
    matrix.append(percentage_to_range)
#print(matrix)


In [ ]:
# check mse per dataset row

from statistics import mean 

mean_for_predicted_dataset = []

for predicted_dataset in matrix:
    mean_for_predicted_dataset.append(mean(predicted_dataset))

print("\n")
for entry in mean_for_predicted_dataset:
    print(entry)

print("Mean:")
print(mean(mean_for_predicted_dataset))



0.6416120476889968
1.6386085155394856
0.08691074023497959
0.02974677535579845
0.054793438092831125
0.05560680289239114
0.228661600450035
0.218461201763019
0.6259961473967948
0.02223263268190794
0.1491603938797818
2.5617789498437435
Mean:
0.5261307704849804


In [ ]:
# Ceck Accuracy among features of test data

#print(y_pred.shape)

y_pred_t = np.transpose(y_pred)
y_test_t = np.transpose(y_test)

# print(y_pred_t.shape)
# print(y_test_t.shape)
featurewise_mean = []
for feature_t, feature_p in zip(y_test_t, y_pred_t):
    # print(feature_t)
    # print(feature_p)
    # print(mean_squared_error(feature_t, feature_p))
    featurewise_mean.append(mean_squared_error(feature_t, feature_p))

print(featurewise_mean)

#print(y_range)
featurewise_mean_percentage = featurewise_mean / y_max**2
print("mse per feature in percentage to value range")
print(featurewise_mean_percentage)

[1518.7794968111255, 0.4944098757409659, 6.181859571268497, 2976.5326859449583, 15.461706919483072]
mse per feature in percentage to value range
NOx                  0.002429
PM 1                 0.002599
CO2                  0.000232
PM 2                 0.000098
Pressure cylinder    0.000591
dtype: Float64


In [ ]:
model.eval()
x = torch.tensor([16.4712565906533, 37.609734587256995, 900.1427871055062, 228.42045796100746, -8.592515890505748, 1981.8799610478413, 3509.5803914574226, 48.709871706251796])
testResults = model(x)
print(type(testResults))
testResults = scaler_y.inverse_transform(testResults.unsqueeze(0).detach().numpy())
print(testResults)
#[125.10746278858385, 0.4371585551795759, 20.70223941458411, 3.4443235692265173, 102.99879114216813]

<class 'torch.Tensor'>


ValueError: non-broadcastable output operand with shape (5,1) doesn't match the broadcast shape (5,5)